# Assignment 15 Solutions

#### 1.	Deep Learning.
- a.	Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.
- b.	Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.
- c.	Tune the hyperparameters using cross-validation and see what precision you can achieve.
- d.	Now try adding Batch Normalization and compare the learning curves: is it converging faster than before? Does it produce a better model?
- e.	Is the model overfitting the training set? Try adding dropout to every layer and try again. Does it help?

In [1]:
import tensorflow as tf
from tensorflow import keras

# Load MNIST dataset
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Filter data for digits 0 to 4
filter_indices = (y_train_full <= 4)
X_train, y_train = X_train_full[filter_indices], y_train_full[filter_indices]

# Preprocess data
X_train, X_test = X_train / 255.0, X_test / 255.0

# Build the DNN
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(5, activation="softmax")
])

# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])

# Define callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model.h5")

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_split=0.1,
                    callbacks=[early_stopping_cb, model_checkpoint_cb])

# Load the best model
model = keras.models.load_model("my_model.h5")

# Evaluate the model on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print test accuracy
print("Test accuracy:", test_accuracy)


Epoch 1/100
861/861 [==============================] - 5s 4ms/step - loss: 0.1148 - accuracy: 0.9640 - val_loss: 0.0474 - val_accuracy: 0.9859
Epoch 2/100
861/861 [==============================] - 3s 3ms/step - loss: 0.0511 - accuracy: 0.9842 - val_loss: 0.0463 - val_accuracy: 0.9869
Epoch 3/100
861/861 [==============================] - 3s 3ms/step - loss: 0.0421 - accuracy: 0.9868 - val_loss: 0.0367 - val_accuracy: 0.9902
Epoch 4/100
861/861 [==============================] - 3s 3ms/step - loss: 0.0306 - accuracy: 0.9900 - val_loss: 0.0431 - val_accuracy: 0.9892
Epoch 5/100
861/861 [==============================] - 3s 3ms/step - loss: 0.0265 - accuracy: 0.9915 - val_loss: 0.0415 - val_accuracy: 0.9915
Epoch 6/100
861/861 [==============================] - 3s 4ms/step - loss: 0.0232 - accuracy: 0.9921 - val_loss: 0.0355 - val_accuracy: 0.9918
Epoch 7/100
861/861 [==============================] - 3s 4ms/step - loss: 0.0217 - accuracy: 0.9931 - val_loss: 0.0396 - val_accuracy: 0.9912

#### 2.	Transfer learning.
- a.	Create a new DNN that reuses all the pretrained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a new one.
- b.	Train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision?
- c.	Try caching the frozen layers, and train the model again: how much faster is it now?
- d.	Try again reusing just four hidden layers instead of five. Can you achieve a higher precision?
- e.	Now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?

In [3]:
# Load the previously trained model (Part 1)
pretrained_model = keras.models.load_model("my_model.h5")

# Freeze the pretrained hidden layers
for layer in pretrained_model.layers[:-1]:
    layer.trainable = False

# Create a new output layer for the transfer learning task
transfer_output = keras.layers.Dense(5, activation="softmax")(pretrained_model.layers[-2].output)

# Create the transfer learning model
transfer_model = keras.Model(inputs=pretrained_model.input, outputs=transfer_output)

# Compile the transfer model
transfer_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer=keras.optimizers.Adam(),
                       metrics=["accuracy"])

# Load digits 5 to 9 from MNIST
(X_train_transfer, y_train_transfer), (X_test_transfer, y_test_transfer) = mnist.load_data()
filter_indices_transfer = (y_train_transfer >= 5)
X_train_transfer, y_train_transfer = X_train_transfer[filter_indices_transfer], y_train_transfer[filter_indices_transfer]
X_train_transfer, X_test_transfer = X_train_transfer / 255.0, X_test_transfer / 255.0

# Train the transfer model
transfer_history = transfer_model.fit(X_train_transfer, y_train_transfer, epochs=100,
                                      validation_split=0.1)

# Evaluate the transfer model on test set
test_loss_transfer, test_accuracy_transfer = transfer_model.evaluate(X_test_transfer, y_test_transfer)

# Print test accuracy
print("Transfer model test accuracy:", test_accuracy_transfer)


Epoch 1/100
827/827 [==============================] - 3s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/100
827/827 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/100
827/827 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/100
827/827 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/100
827/827 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/100
827/827 [==============================] - 3s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/100
827/827 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accura

827/827 [==============================] - 4s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 58/100
827/827 [==============================] - 4s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 59/100
827/827 [==============================] - 4s 4ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 60/100
827/827 [==============================] - 4s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 61/100
827/827 [==============================] - 4s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 62/100
827/827 [==============================] - 4s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 63/100
827/827 [==============================] - 4s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.

#### 3.	Pretraining on an auxiliary task.
- a.	In this exercise you will build a DNN that compares two MNIST digit images and predicts whether they represent the same digit or not. Then you will reuse the lower layers of this network to train an MNIST classifier using very little training data. Start by building two DNNs (let’s call them DNN A and B), both similar to the one you built earlier but without the output layer: each DNN should have five hidden layers of 100 neurons each, He initialization, and ELU activation. Next, add one more hidden layer with 10 units on top of both DNNs. To do this, you should use TensorFlow’s concat() function with axis=1 to concatenate the outputs of both DNNs for each instance, then feed the result to the hidden layer. Finally, add an output layer with a single neuron using the logistic activation function.
- b.	Split the MNIST training set in two sets: split #1 should containing 55,000 images, and split #2 should contain contain 5,000 images. Create a function that generates a training batch where each instance is a pair of MNIST images picked from split #1. Half of the training instances should be pairs of images that belong to the same class, while the other half should be images from different classes. For each pair, the training label should be 0 if the images are from the same class, or 1 if they are from different classes.
- c.	Train the DNN on this training set. For each image pair, you can simultaneously feed the first image to DNN A and the second image to DNN B. The whole network will gradually learn to tell whether two images belong to the same class or not.
- d.	Now create a new DNN by reusing and freezing the hidden layers of DNN A and adding a softmax output layer on top with 10 neurons. Train this network on split #2 and see if you can achieve high performance despite having only 500 images per class.

In [4]:
import tensorflow as tf
from tensorflow import keras

# Load MNIST dataset
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Preprocess data
X_train_full, X_test = X_train_full / 255.0, X_test / 255.0

# Define the architecture for DNN A and DNN B
dnn_a = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="elu", kernel_initializer="he_normal")
])

dnn_b = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="elu", kernel_initializer="he_normal")
])

# Create the concatenated hidden layer for auxiliary task
concatenated = keras.layers.concatenate([dnn_a.output, dnn_b.output])
hidden_layer_aux = keras.layers.Dense(10, activation="elu", kernel_initializer="he_normal")(concatenated)

# Create the auxiliary task model
auxiliary_model = keras.Model(inputs=[dnn_a.input, dnn_b.input], outputs=hidden_layer_aux)

# Compile the auxiliary model
auxiliary_model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam())

# Split MNIST training set into two sets
split_1_indices = (y_train_full < 5)
X_train_split_1, y_train_split_1 = X_train_full[split_1_indices], y_train_full[split_1_indices]

# Function to generate training batch for auxiliary task
def generate_batch(X, batch_size):
    batch_indices = tf.random.uniform(shape=[batch_size], maxval=len(X), dtype=tf.int32)
    X_batch = X[batch_indices]
    y_batch = tf.random.uniform(shape=[batch_size], maxval=2, dtype=tf.float32)
    is_same_digit = tf.equal(y_batch, 0.0)
    for i in range(batch_size):
        if is_same_digit[i]:
            y_batch[i] = tf.cast(tf.equal(X_batch[i, 0], X_batch[i, 1]), tf.float32)
        else:
            y_batch[i] = 1.0
    return [X_batch[:, 0], X_batch[:, 1]], y_batch

# Train the auxiliary model on the auxiliary task
batch_size = 32
auxiliary_model.fit(generate_batch(X_train_split_1, batch_size), epochs=10)

# Create a new model reusing and freezing DNN A's hidden layers
dnn_a_clone = keras.models.clone_model(dnn_a)
dnn_a_clone.layers.pop()  # Remove the last layer
transfer_output = dnn_a_clone.layers[-1].output
transfer_model = keras.Model(inputs=dnn_a_clone.input, outputs=transfer_output)

# Freeze the hidden layers of DNN A clone
for layer in transfer_model.layers[:-1]:
    layer.trainable = False

# Add a softmax output layer for classification
output_layer_transfer = keras.layers.Dense(10, activation="softmax")(transfer_output)
transfer_model = keras.Model(inputs=transfer_model.input, outputs=output_layer_transfer)

# Compile the transfer model
transfer_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer=keras.optimizers.Adam(),
                       metrics=["accuracy"])

# Split MNIST training set #2
split_2_indices = (y_train_full >= 5)
X_train_split_2, y_train_split_2 = X_train_full[split_2_indices], y_train_full[split_2_indices]

# Train the transfer model on split #2
transfer_model.fit(X_train_split_2, y_train_split_2, epochs=10)


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment